In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option("display.max_columns", None)

train_df = pd.read_csv("../input/tabular-playground-series-dec-2021/train.csv")
test_df = pd.read_csv("../input/tabular-playground-series-dec-2021/test.csv")
sub_df = pd.read_csv("../input/tabular-playground-series-dec-2021/sample_submission.csv")

train_df.head()

In [ ]:
train_df.groupby("Cover_Type")['Id'].nunique()

In [ ]:
temp_df=train_df[train_df.Cover_Type==5]

In [ ]:
temp_df

In [ ]:
temp_df.shape

In [ ]:
train_df=train_df.append(temp_df)

In [ ]:
train_df.shape

PREPROCESSNG BROUGHT FROM BIBHABASU MOHAPATRA https://www.kaggle.com/bibhabasumohapatra/multiclass-using-predict-proba-ensemble-part-2

In [ ]:
cols = ["Soil_Type7", "Soil_Type15"]

train_df.drop(cols, axis=1, inplace=True)
test_df.drop(cols, axis=1, inplace=True)

In [ ]:
new_names = {
    "Horizontal_Distance_To_Hydrology": "x_dist_hydrlgy",
    "Vertical_Distance_To_Hydrology": "y_dist_hydrlgy",
    "Horizontal_Distance_To_Roadways": "x_dist_rdwys",
    "Horizontal_Distance_To_Fire_Points": "x_dist_firepts"
}

train_df.rename(new_names, axis=1, inplace=True)
test_df.rename(new_names, axis=1, inplace=True)

In [ ]:
train_df["Cover_Type"]=train_df["Cover_Type"]-1

In [ ]:
train_df.groupby("Cover_Type")['Id'].nunique()

In [ ]:
train_df["Aspect"][train_df["Aspect"] < 0] += 360
train_df["Aspect"][train_df["Aspect"] > 359] -= 360

test_df["Aspect"][test_df["Aspect"] < 0] += 360
test_df["Aspect"][test_df["Aspect"] > 359] -= 360

In [ ]:
train_df["mnhttn_dist_hydrlgy"] = np.abs(train_df["x_dist_hydrlgy"]) + np.abs(train_df["y_dist_hydrlgy"])
test_df["mnhttn_dist_hydrlgy"] = np.abs(test_df["x_dist_hydrlgy"]) + np.abs(test_df["y_dist_hydrlgy"])

# Euclidean distance to Hydrology
train_df["ecldn_dist_hydrlgy"] = (train_df["x_dist_hydrlgy"]**2 + train_df["y_dist_hydrlgy"]**2)**0.5
test_df["ecldn_dist_hydrlgy"] = (test_df["x_dist_hydrlgy"]**2 + test_df["y_dist_hydrlgy"]**2)**0.5

In [ ]:
soil_features = [x for x in train_df.columns if x.startswith("Soil_Type")]
train_df["soil_type_count"] = train_df[soil_features].sum(axis=1)
test_df["soil_type_count"] = test_df[soil_features].sum(axis=1)

wilderness_features = [x for x in train_df.columns if x.startswith("Wilderness_Area")]
train_df["wilderness_area_count"] = train_df[wilderness_features].sum(axis=1)
test_df["wilderness_area_count"] = test_df[wilderness_features].sum(axis=1)

In [ ]:
train_df.loc[train_df["Hillshade_9am"] < 0, "Hillshade_9am"] = 0
test_df.loc[test_df["Hillshade_9am"] < 0, "Hillshade_9am"] = 0

train_df.loc[train_df["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0
test_df.loc[test_df["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0

train_df.loc[train_df["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0
test_df.loc[test_df["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0

train_df.loc[train_df["Hillshade_9am"] > 255, "Hillshade_9am"] = 255
test_df.loc[test_df["Hillshade_9am"] > 255, "Hillshade_9am"] = 255

train_df.loc[train_df["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255
test_df.loc[test_df["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255

train_df.loc[train_df["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255
test_df.loc[test_df["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255

In [ ]:
from sklearn.preprocessing import RobustScaler


cols = [
    "Elevation",
    "Aspect",
    "mnhttn_dist_hydrlgy",
    "ecldn_dist_hydrlgy",
    "soil_type_count",
    "wilderness_area_count",
    "Slope",
    "x_dist_hydrlgy",
    "y_dist_hydrlgy",
    "x_dist_rdwys",
    "Hillshade_9am",
    "Hillshade_Noon",
    "Hillshade_3pm",
    "x_dist_firepts",
    "soil_type_count",
    "wilderness_area_count"
]

scaler = RobustScaler()
train_df[cols] = scaler.fit_transform(train_df[cols])
test_df[cols] = scaler.transform(test_df[cols])

In [ ]:
df=train_df

In [ ]:
df.groupby("Cover_Type")['Id'].nunique()

In [ ]:
df.rename({'Cover_Type':'target'}, axis=1, inplace=True)

In [ ]:
df.groupby("target")['Id'].nunique()

In [ ]:
from sklearn import model_selection
df["kfold"] = -1

df = df.sample(frac=1).reset_index(drop=True)

kf = model_selection.StratifiedKFold(n_splits=2, shuffle=False)

for fold, (train_idx, val_idx) in enumerate(kf.split(X=df, y=df.target.values)):
    print(len(train_idx), len(val_idx))
    df.loc[val_idx, 'kfold'] = fold

In [ ]:
df.groupby("target")['Id'].nunique()

In [ ]:
df.to_csv("tpsDecemberTrainingDataPreprocessed_new.csv",index=False)
test_df.to_csv("tpsDecemberTestingDataPreprocessed_new.csv",index=False)

TRAINING XGBOOST

In [ ]:
n_folds=2

In [ ]:
import joblib
import xgboost as xgb

In [ ]:
df_folds=pd.read_csv("../input/preprocessedtrainingandtestingdata-new/prprocessedTrainingAndTestingData/tpsDecemberTrainingDataPreprocessed_new.csv")

In [ ]:
def run_folds(df,fold):
    df_train=df[df.kfold!=fold]
    df_test=df[df.kfold==fold]
    print("Entered")
    param={'tree_method':'gpu_hist'}
    
    clf= xgb.XGBClassifier(**param)
    clf.fit(df_train.drop(['target','Id','kfold'],axis=1).values,df_train.target.values)
    
    y_pred = clf.predict_proba(df_test.drop(['target','Id','kfold'],axis=1).values)
    
    File_name = 'model_xgbclf' + str(fold)
    joblib.dump(
    clf,File_name)
    print(y_pred.shape)
    df_test.loc[:,"xgbclf_pred1"]=y_pred[:,0]
    df_test.loc[:,"xgbclf_pred2"]=y_pred[:,1]
    df_test.loc[:,"xgbclf_pred3"]=y_pred[:,2]
    df_test.loc[:,"xgbclf_pred4"]=y_pred[:,3]
    df_test.loc[:,"xgbclf_pred5"]=y_pred[:,4]
    df_test.loc[:,"xgbclf_pred6"]=y_pred[:,5]
    df_test.loc[:,"xgbclf_pred7"]=y_pred[:,6]
    
    return df_test[['Id','target','kfold','xgbclf_pred1','xgbclf_pred2','xgbclf_pred3','xgbclf_pred4','xgbclf_pred5','xgbclf_pred6','xgbclf_pred7']]


dfs=[]
    
for i in range(n_folds):
    temp_df=run_folds(df_folds,i)
    dfs.append(temp_df)
df_pred=pd.concat(dfs)

print(df_pred.shape)

In [ ]:
df_pred.to_csv("XGBOOST_PREDICTIONS_PREPROCESSED_NEW.csv",index=False)

LGBM TRAINING

In [ ]:
n_folds=2

In [ ]:
import joblib
import lightgbm as lgb

In [ ]:
df_folds=pd.read_csv("./tpsDecemberTrainingDataPreprocessed_new.csv")

In [ ]:
df_folds.groupby('target')['Id'].nunique()

In [ ]:
def run_folds(df,fold):
    print("ENTER")
    df_train=df[df.kfold!=fold]
    df_test=df[df.kfold==fold]
    
    clf= lgb.LGBMClassifier()
    clf.fit(df_train.drop(['target','Id','kfold'],axis=1).values,df_train.target.values)
    
    y_pred = clf.predict_proba(df_test.drop(['target','Id','kfold'],axis=1).values)#[:,1]

    
    File_name = 'model_lgbmclf' + str(fold)
    joblib.dump(
    clf,File_name)
    print(y_pred.shape)
    df_test.loc[:,"lgbmclf_pred1"]=y_pred[:,0]
    df_test.loc[:,"lgbmclf_pred2"]=y_pred[:,1]
    df_test.loc[:,"lgbmclf_pred3"]=y_pred[:,2]
    df_test.loc[:,"lgbmclf_pred4"]=y_pred[:,3]
    df_test.loc[:,"lgbmclf_pred5"]=y_pred[:,4]
    df_test.loc[:,"lgbmclf_pred6"]=y_pred[:,5]
    df_test.loc[:,"lgbmclf_pred7"]=y_pred[:,6]
    
    return df_test[['Id','target','kfold','lgbmclf_pred1','lgbmclf_pred2','lgbmclf_pred3','lgbmclf_pred4','lgbmclf_pred5','lgbmclf_pred6','lgbmclf_pred7']]


dfs=[]
    
for i in range(n_folds):
    temp_df=run_folds(df_folds,i)
    dfs.append(temp_df)
df_pred=pd.concat(dfs)

print(df_pred.shape)

In [ ]:
df_pred.head()

In [ ]:
df_pred.groupby('target')['Id'].nunique()

In [ ]:
df_pred.to_csv("LGBM_PRED_PREPROCESSED_NEW.csv",index=False)

CHECKING ACCURACY OF LGBM

In [ ]:
df_lgbm=pd.read_csv("./LGBM_PRED_PREPROCESSED_NEW.csv")

In [ ]:
df_lgbm.head()

In [ ]:
arr=np.array(df_lgbm.iloc[:,3:])

In [ ]:
df_lgbm["pred_class"]=arr.argmax(axis=1)+1

In [ ]:
df_lgbm.head()

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy of LGBM: ",end="")
print(accuracy_score(df_lgbm['target']+1,df_lgbm["pred_class"]))

CHECKIN ACCURACY OF XGBOOST

In [ ]:
df_xgb=pd.read_csv("./XGBOOST_PREDICTIONS_PREPROCESSED_NEW.csv")
df_xgb.head()

In [ ]:
arr=np.array(df_xgb.iloc[:,3:])
df_xgb["pred_class"]=arr.argmax(axis=1)+1
from sklearn.metrics import accuracy_score
print("Accuracy of XGB: ",end="")
print(accuracy_score(df_xgb['target']+1,df_xgb["pred_class"]))

### TRAINGING CATBOOST BY STACKING LGBM AND XGBOOST

In [ ]:
df_folds=pd.read_csv("../input/preprocessedtrainingandtestingdata-new/prprocessedTrainingAndTestingData/tpsDecemberTrainingDataPreprocessed_new.csv")

In [ ]:
df_xgb=pd.read_csv("../input/xgboostpreprocesseddecember-new/xgboostPreprocessedDecember_NEW/XGBOOST_PREDICTIONS_PREPROCESSED_NEW.csv")
df_lgbm=pd.read_csv("../input/lgbmpreprocesseddecembertpsnew/lgbmPreprocessedNew/LGBM_PRED_PREPROCESSED_NEW.csv")

In [ ]:
df_lgbm.head()

In [ ]:
df_xgb.head()

In [ ]:
df_lgbm2=df_lgbm.iloc[:,3:]

In [ ]:
df_lgbm2.loc[:,'Id']=df_lgbm.loc[:,'Id']

In [ ]:
df_lgbm2

In [ ]:
df_xgb2=pd.merge(df_xgb,df_lgbm2,on='Id')

In [ ]:
df_xgb2.shape,df_xgb.shape,df_lgbm.shape,df_lgbm2.shape

In [ ]:
df_xgb2.head()

In [ ]:
df_xgb3=df_xgb2.iloc[:,3:]
df_xgb3.loc[:,"Id"]=df_xgb2.loc[:,"Id"]
df_xgb3.head()

In [ ]:
df_folds1=pd.merge(df_folds,df_xgb3,on='Id')
df_folds1.shape,df_folds.shape,df_xgb3.shape

In [ ]:
df_folds1.to_csv("CB_FOLDS.csv",index=False)

In [ ]:
n_folds=2

In [ ]:
import joblib
import catboost as cb

In [ ]:
df_folds1.columns

In [ ]:
def run_folds(df,fold):
    print("ENTER")
    df_train=df[df.kfold!=fold]
    df_test=df[df.kfold==fold]
    
    clf= cb.CatBoostClassifier()
    clf.fit(df_train.drop(['target','Id','kfold'],axis=1).values,df_train.target.values)
    
    y_pred = clf.predict_proba(df_test.drop(['target','Id','kfold'],axis=1).values)#[:,1]

    
    File_name = 'model_cbclf' + str(fold)
    joblib.dump(
    clf,File_name)
    print(y_pred.shape)
    df_test.loc[:,"cbclf_pred1"]=y_pred[:,0]
    df_test.loc[:,"cbclf_pred2"]=y_pred[:,1]
    df_test.loc[:,"cbclf_pred3"]=y_pred[:,2]
    df_test.loc[:,"cbclf_pred4"]=y_pred[:,3]
    df_test.loc[:,"cbclf_pred5"]=y_pred[:,4]#-y_pred[:,4]
    df_test.loc[:,"cbclf_pred6"]=y_pred[:,5]
    df_test.loc[:,"cbclf_pred7"]=y_pred[:,6]
    
    return df_test[['Id','target','kfold','cbclf_pred1','cbclf_pred2','cbclf_pred3','cbclf_pred4','cbclf_pred5','cbclf_pred6','cbclf_pred7']]


dfs=[]
    
for i in range(n_folds):
    temp_df=run_folds(df_folds1,i)
    dfs.append(temp_df)
df_pred=pd.concat(dfs)

print(df_pred.shape)

In [ ]:
df_pred.to_csv("CB_PRED_NEW.csv",index=False)

In [ ]:
df_xgb2.to_csv("CB_PART_OF_TRAINING_DATA_NEW.csv",index=False)

### FINALLY MAKING PREDICTIONS

In [ ]:
df_test=pd.read_csv("../input/preprocessedtrainingandtestingdata-new/prprocessedTrainingAndTestingData/tpsDecemberTestingDataPreprocessed_new.csv")

In [ ]:
import joblib
xgb0=joblib.load("../input/xgboostpreprocesseddecember-new/xgboostPreprocessedDecember_NEW/model_xgbclf0 (3)")
xgb1=joblib.load("../input/xgboostpreprocesseddecember-new/xgboostPreprocessedDecember_NEW/model_xgbclf1 (4)")
y_pred0=xgb0.predict_proba(df_test.drop(columns=['Id']))
y_pred1=xgb1.predict_proba(df_test.drop(columns=['Id']))
y_pred_xgb=(y_pred0+y_pred1)/2

In [ ]:
import joblib
lgbm0=joblib.load("../input/lgbmpreprocesseddecembertpsnew/lgbmPreprocessedNew/model_lgbmclf0 (3)")
lgbm1=joblib.load("../input/lgbmpreprocesseddecembertpsnew/lgbmPreprocessedNew/model_lgbmclf1 (3)")
y_pred0=lgbm0.predict_proba(df_test.drop(columns=['Id']))
y_pred1=lgbm1.predict_proba(df_test.drop(columns=['Id']))
y_pred=(y_pred0+y_pred1)/2

In [ ]:
df_test.loc[:,"xgbclf_pred1"]=y_pred_xgb[:,0]
df_test.loc[:,"xgbclf_pred2"]=y_pred_xgb[:,1]
df_test.loc[:,"xgbclf_pred3"]=y_pred_xgb[:,2]
df_test.loc[:,"xgbclf_pred4"]=y_pred_xgb[:,3]
df_test.loc[:,"xgbclf_pred5"]=y_pred_xgb[:,4]#-y_pred_xgb[:,4]
df_test.loc[:,"xgbclf_pred6"]=y_pred_xgb[:,5]
df_test.loc[:,"xgbclf_pred7"]=y_pred_xgb[:,6]

In [ ]:
df_test.loc[:,"lgbmclf_pred1"]=y_pred[:,0]
df_test.loc[:,"lgbmclf_pred2"]=y_pred[:,1]
df_test.loc[:,"lgbmclf_pred3"]=y_pred[:,2]
df_test.loc[:,"lgbmclf_pred4"]=y_pred[:,3]
df_test.loc[:,"lgbmclf_pred5"]=y_pred[:,4]#-y_pred[:,4]
df_test.loc[:,"lgbmclf_pred6"]=y_pred[:,5]
df_test.loc[:,"lgbmclf_pred7"]=y_pred[:,6]

In [ ]:
df_test.head()

In [ ]:
df_test.columns

In [ ]:
df_test.to_csv("pass_through_catboost_preprocessed_NEW.csv",index=False)

In [ ]:
import joblib
cb0=joblib.load("../input/catboost-model-december-new/cbmodelDecemberNew/model_cbclf0")
cb1=joblib.load("../input/catboost-model-december-new/cbmodelDecemberNew/model_cbclf1")
y_pred0=cb0.predict_proba(df_test.drop(columns=['Id']))
y_pred1=cb1.predict_proba(df_test.drop(columns=['Id']))
y_pred_cb=(y_pred0+y_pred1)/2

In [ ]:
clas=y_pred_cb.argmax(axis=1)+1

In [ ]:
sub=pd.read_csv("../input/tabular-playground-series-dec-2021/sample_submission.csv")

In [ ]:
sub['Cover_Type']=clas

In [ ]:
sub.to_csv("STACKED_SUBMISSION_NEW.csv",index=False)

In [ ]:
y_PRED=(y_pred_cb+y_pred_xgb+y_pred)/3
clas=y_PRED.argmax(axis=1)+1
sub['Cover_Type']=clas
sub.to_csv("STACKED_AND_ENSEMBLED3_SUBMISSION_NEW.csv",index=False)

In [ ]:
y_PRED=(y_pred_cb+y_pred_xgb)/2
clas=y_PRED.argmax(axis=1)+1
sub['Cover_Type']=clas
sub.to_csv("STACKED_AND_ENSEMBLED2_SUBMISSION_NEW.csv",index=False)  #This is best score 